In [ ]:
SCOPETYPE = "OPENADC"
PLATFORM = "CWHUSKY"

%run "../Setup_Generic.ipynb"


import chipwhisperer as cw
from chipwhisperer.capture.scopes.OpenADC import OpenADC
from chipwhisperer.capture.targets.SimpleSerial2 import SimpleSerial2
from chipwhisperer.capture.api.programmers import XMEGAProgrammer
import matplotlib.pyplot as plt
import time
from pprint import pprint
import numpy as np
from typing import Optional, List, Set, Any
print(type(scope))
print(type(target))
print(type(prog))
scope: OpenADC = scope
target: SimpleSerial2 = target
prog: XMEGAProgrammer = prog

print("Scope:\n", scope)

In [ ]:

refresh_rate = 200e6

scope.adc.timeout = 5
scope.adc.samples = 118_000

print("Setting clkgen_src")
scope.clock.clkgen_src = "system" 
print("Set clkgen_src")

print("Setting refresh rate")
scope.clock.clkgen_freq = refresh_rate
print("set refresh rate")

print("Setting adc_mul")
scope.clock.adc_mul = 1
print("Set adc_mul")

print("'Real freq': scope.clock.adc_freq:", scope.clock.adc_freq)
if scope.clock.adc_freq is not None:
    print(scope.clock.adc_freq / 1e6, "MHz")


print("scope.clock.adc_mul:", scope.clock.adc_mul)
print("scope.clock.clkgen_freq", scope.clock.clkgen_freq)


In [ ]:
capacitor = "TP15"
category = "TODO"

# ******
# Do this to discard the first trace
scope.arm()
ret = scope.capture()
if ret:
    print("Timeout - no trigger seen")

scope.get_last_trace()
# ******

traces = []

successful_traces = 0
i = 0

while successful_traces < 300:
    i += 1

    scope.arm()
    ret = scope.capture()
    if ret:
        print("Timeout - no trigger seen")
        continue
    
    trace = scope.get_last_trace()

    traces.append(trace)
    print(f"Iter: {i} - The trigger was hit on sample:", scope.adc.trig_count)
    successful_traces += 1


In [ ]:
%matplotlib ipympl
plt.figure()
for t in traces[:50]:
    plt.plot(t, alpha=0.6)

plt.show()


In [ ]:
print(traces)

import os
import csv

output_dir = f"../../traces/{category}/{capacitor}/"

os.makedirs(output_dir, exist_ok=True)

for i, trace in enumerate(traces):
    filename = os.path.join(output_dir, f"trace_{i+1}.csv")
    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        for value in trace:
            writer.writerow([value])

print(f"Saved {len(traces)} trace files in {output_dir}")


In [ ]:
plt.figure()
for t in traces:
    plt.plot(t)


directory = f"../../images/{category}/"
filename = f"{capacitor}.png"
filepath = os.path.join(directory, filename)

os.makedirs(directory, exist_ok=True)

plt.savefig(filepath)
plt.show()

In [ ]:
scope.dis()
target.dis()